In [2]:
import pandas as pd 
pd.set_option('display.max_columns', None)
caminho_do_arquivo = r"C:\curso-eng-dados\Python-SGBDS\Arquivos/Origem de dados/V_OCORRENCIA_AMPLA.json"
df = pd.read_json(caminho_do_arquivo, encoding='utf-8-sig')

colunas = ["Numero_da_Ocorrencia", "Classificacao_da_Ocorrência", "Data_da_Ocorrencia","Municipio","UF","Regiao","Nome_do_Fabricante","Modelo"]
df = df[colunas]
df.rename( columns={  'Classificacao_da_Ocorrência' : 'Classificacao_da_Ocorrencia'  } ,inplace=True )
df.head()

,Numero_da_Ocorrencia,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Municipio,UF,Regiao,Nome_do_Fabricante,Modelo
0,7762,Incidente,2018-03-21,SÃO PAULO,SP,Sudeste,AGUSTA,AW109SP
1,7759,Acidente,2018-03-14,MONTES CLAROS,MG,Sudeste,CESSNA AIRCRAFT,A152
2,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT,T188C
3,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT,T188C
4,7757,Incidente Grave,2018-03-18,TORRES,RS,Sul,PIPER AIRCRAFT,PA-34-200


In [8]:
df.dtypes

Numero_da_Ocorrencia                    int64
Classificacao_da_Ocorrencia            object
Data_da_Ocorrencia             datetime64[ns]
Municipio                              object
UF                                     object
Regiao                                 object
Nome_do_Fabricante                     object
Modelo                                 object
dtype: object

In [ ]:
df['Data_da_Ocorrencia'] = pd.to_datetime(df['Data_da_Ocorrencia'])


,Numero_da_Ocorrencia,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Municipio,UF,Regiao,Nome_do_Fabricante,Modelo
0,7762,Incidente,2018-03-21,SÃO PAULO,SP,Sudeste,AGUSTA,AW109SP
1,7759,Acidente,2018-03-14,MONTES CLAROS,MG,Sudeste,CESSNA AIRCRAFT,A152
2,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT,T188C
3,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT,T188C
4,7757,Incidente Grave,2018-03-18,TORRES,RS,Sul,PIPER AIRCRAFT,PA-34-200
5,5552,Acidente,2009-07-07,None,PR,Sul,None,None
6,5118,Acidente,2006-10-13,None,ES,Sudeste,PIPER AIRCRAFT,PA-34-200
7,7756,Incidente Grave,2018-03-18,IGARASSÚ,PE,Nordeste,KAIO AMARAL RANGEL,RV-7
8,7755,Incidente Grave,2018-03-18,MARAGOGI,AL,Nordeste,HENRIQUE DRUMOND LIMA DE OLIVEIRA,PARADISE P-1
9,7754,Incidente Grave,2018-03-17,BRAGANÇA PAULISTA,SP,Sudeste,NEIVA,56-C


In [9]:
from datetime import datetime

ano_atual = datetime.now().year
print(ano_atual)

2025


In [11]:
df = df[df['Data_da_Ocorrencia'].dt.year == ano_atual]
df.head(10)

,Numero_da_Ocorrencia,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Municipio,UF,Regiao,Nome_do_Fabricante,Modelo


In [ ]:
from sqlalchemy import create_engine ,Integer,String,Date,VARCHAR,text

dbname   = 'python'
user     = 'postgres'
password = '*********'
host     = 'localhost'
port     = '5432' 

conexao_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(conexao_str)

nome_tabela = 'anac_sqlalchemy'

#Deletar registros com base no ano atual
cursor=engine.connect()
delete = text(f'DELETE FROM puclib.{nome_tabela} WHERE EXTRACT(YEAR FROM "Data_da_Ocorrencia") = {ano_atual}')
cursor.execute(delete)
cursor.commit()

# Enviar DataFrama para o BD
df.to_sql(nome_tabela,engine,index=False, if_exists='replace',dtype={

'Numero_da_Ocorrencia' : Integer ,
'Classificacao_da_Ocorrencia': VARCHAR(50),
'Data_da_Ocorrencia':Date

})

engine.dispose()
#cursor.close()